In [1]:
"""
Initial data exploration in order to figure out appropriate functional form of features and general relationships between variables.
Analysis restricted to patch: 10.9
"""

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import time
import datetime
from config import PATH_GDRIVE_MAIN_DIR

PATCH = 10.9
comps_data = pd.read_pickle(f"{PATH_GDRIVE_MAIN_DIR}{PATCH}_compositions_data.pkl")

In [4]:
comps_data.head()

,Game Version,Game Date,GameID,Region,PUUID,Place,Traits,Units,Damage,TeamSize
0,Version 10.8.317.8137 (Apr 17 2020/16:09:11) [...,2020-04-28 02:08:53.789,BR1_1928959955,BR1,LnhDH7pkBkkJJPX_KtclNy549cwmuw15LP9oNihVTEQ997...,3,"[Blaster1, Chrono2, ManaReaver1, Mercenary1, S...","[{'name': 'TFT3_Blitzcrank', 'tier': 2, 'items...",114,8
1,Version 10.8.317.8137 (Apr 17 2020/16:09:11) [...,2020-04-28 02:08:53.789,BR1_1928959955,BR1,Ku6me57MBtrKUZcyBLpAcoR_4Nljt02-pTSyKECbuN54OS...,5,"[ManaReaver1, Protector1, Set3_Celestial2, Spa...","[{'name': 'TFT3_Darius', 'tier': 3, 'items': [...",92,8
2,Version 10.8.317.8137 (Apr 17 2020/16:09:11) [...,2020-04-28 02:08:53.789,BR1_1928959955,BR1,lVWTKKrRMc---TIo_ikG4nak6e0uxG5zglYb23TxVb0tkQ...,4,"[Blaster1, Chrono2, Cybernetic1, ManaReaver1, ...","[{'name': 'TFT3_Fiora', 'tier': 1, 'items': [1...",84,8
3,Version 10.8.317.8137 (Apr 17 2020/16:09:11) [...,2020-04-28 02:08:53.789,BR1_1928959955,BR1,eaZTaLRRnDFMlTrjK6RSWYR8S2Xbw-giGC-Dh_D3NqmCr4...,2,"[Demolitionist1, Infiltrator2, MechPilot1, Val...","[{'name': 'Khazix', 'tier': 2, 'items': [1]}, ...",184,7
4,Version 10.8.317.8137 (Apr 17 2020/16:09:11) [...,2020-04-28 02:08:53.789,BR1_1928959955,BR1,Rq-_KV48ZDQadUDOgGtFaenrYZ_LCd2y4kZhQF7TCbp12x...,6,"[Blaster2, Mercenary1, Set3_Blademaster1, Set3...","[{'name': 'TFT3_Fiora', 'tier': 1, 'items': []...",111,9


In [13]:
# Need to remove some samples from the previous game version 
comps_data = comps_data.loc[comps_data['Game Version'].str.extract(r'(\d*[.][^.]*)', expand=False) == "10.9",].reset_index()
comps_data['Game Version'].unique()

# Save cleaned df
comps_data.to_csv(f"{PATH_GDRIVE_MAIN_DIR}10.9_compositions_data.pkl")

In [17]:
# Isolate analysis to just top 2 places for now 
comps_data_top2 = comps_data.loc[comps_data['Place'].isin([1,2]),].reset_index()

# Flatten lists of traits 
comps_data_traits = comps_data_top2.explode('Traits')

# Flatten lists of unit dicts 
comps_data_units = comps_data_top2.explode('Units')

In [29]:
# Clean traits strings
comps_data_traits.Traits = comps_data_traits.Traits.str.replace("Set3_", "", regex=False)
comps_data_units.Units = comps_data_units.Units.str.replace("TFT3_", "", regex=False)

In [35]:
# Check for duplicated traits within the same game 
sum(comps_data_traits.groupby(['Traits', 'GameID', 'Place']).agg(Counts=('Region', 'count')).Counts != 1)

0

In [48]:
# Compute individual trait winrates
comps_data_traits['Win'] = comps_data_traits['Place'] == 1

indiv_trait_wr = comps_data_traits.groupby('Traits').agg(WR=('Win', lambda x: sum(x)/len(x)),
                                                            N = ('Place','count')).reset_index().sort_values(by='WR', ascending=False)
indiv_trait_wr

,Traits,WR,N
2,Blademaster3,0.717391,46
37,Sorcerer4,0.620178,674
39,SpacePirate2,0.614211,2688
17,DarkStar3,0.611111,108
29,Protector3,0.574713,261
46,Void1,0.570652,9009
25,Mystic1,0.544844,32736
32,Rebel3,0.537037,108
26,Mystic2,0.536180,1686
41,StarGuardian2,0.529264,6783


In [ ]:
# Compute winrates for every 